### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [ ]:
from modelModule.model2 import VAE2
import torch

In [ ]:
model = VAE2()
a = torch.randn((10,57))
b = torch.randn((10,57))
model(a,b)

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = FlatDataset('Spam_test.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
batch.keys()

### 模型精度测定

In [7]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
from modelModule.model2 import VAE2
from torch.utils.data import DataLoader
from utils import restore_data, result_show, get_missing
import torch
import numpy as np

In [3]:
## 模型参数载入
model = VAE2(dim=16, nhead=4)
checkpoint_path = r'模型参数保存\Letter_lr_0_0001_lr_scheduler_CosineAnnealingLR\version_3\checkpoints\epoch=49-step=3150.ckpt'    
checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
print('载入成功')

载入成功


In [12]:
dataset = FlatDataset('Letter_test.csv', is_test=True, data_norm='mean_norm') # minmax_norm
index = 102
# src_data, M_matrix, miss_data = dataset[index]['src_data'].reshape(1,-1), dataset[index]['global_miss_matrix'].reshape(1,-1), dataset[index]['global_miss_data'].reshape(1,-1)
src_data = dataset[index]['src_data'].reshape(1,-1)
normal_data = dataset[index]['global_normal_data'].reshape(1,-1)
miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))
result_show(src_data, res_imputed_data, M_matrix)

原始数据 [ 3.  4. 10.  1.]
插补数据 [4. 5. 8. 3.]


array([0.33333333, 0.25      , 0.2       , 1.99999998])

In [ ]:
normal_data

In [ ]:
imputed_data